# CrewAI Multi-Agent System with Gemini 3 Pro

> **Created by [Build Fast with AI](https://www.buildfastwithai.com)**

This notebook demonstrates building a multi-agent content creation system using CrewAI and Gemini 3 Pro.

## What you'll learn:
- Introduction to CrewAI
- Creating specialized agents
- Agent collaboration and task delegation
- Building a content creation crew
- Process orchestration

In [ ]:
!pip install -q crewai crewai-tools langchain-google-genai

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Markdown, display

In [ ]:
# Configure API key
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY', 'your-api-key-here')

os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

## 1. Initialize Gemini Model for CrewAI

In [ ]:
# Initialize Gemini for agents
llm = ChatGoogleGenerativeAI(
    model="gemini-3-pro",
    temperature=0.7,
    google_api_key=GOOGLE_API_KEY
)

print("Gemini model initialized for CrewAI")

## 2. Create Specialized Agents

Let's create a content creation team with different specialized agents.

In [ ]:
# Research Agent
researcher = Agent(
    role='Content Researcher',
    goal='Research and gather comprehensive information on given topics',
    backstory="""You are an expert researcher with years of experience in 
    gathering and synthesizing information from various sources. You have a 
    keen eye for relevant details and can quickly identify key insights.""",
    llm=llm,
    verbose=True,
    allow_delegation=True
)

# Writer Agent
writer = Agent(
    role='Content Writer',
    goal='Create engaging and well-structured content based on research',
    backstory="""You are a skilled content writer with expertise in crafting 
    compelling narratives. You excel at transforming research into readable, 
    engaging content that resonates with the target audience.""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# Editor Agent
editor = Agent(
    role='Content Editor',
    goal='Review and improve content for clarity, grammar, and impact',
    backstory="""You are a meticulous editor with an eye for detail. You 
    ensure all content is polished, error-free, and maintains a consistent 
    tone and style.""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# SEO Specialist Agent
seo_specialist = Agent(
    role='SEO Specialist',
    goal='Optimize content for search engines and readability',
    backstory="""You are an SEO expert who understands how to optimize 
    content for search engines while maintaining quality and readability. 
    You know the latest SEO best practices and trends.""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

print("Created 4 specialized agents!")

## 3. Define Tasks for Each Agent

In [ ]:
def create_content_tasks(topic: str):
    """Create tasks for the content creation crew."""
    
    # Research Task
    research_task = Task(
        description=f"""Research the topic: {topic}
        
        Gather information about:
        1. Key concepts and definitions
        2. Current trends and developments
        3. Common questions and pain points
        4. Target audience interests
        5. Relevant examples and case studies
        
        Provide a comprehensive research summary.""",
        agent=researcher,
        expected_output="A detailed research summary with key findings and insights"
    )
    
    # Writing Task
    writing_task = Task(
        description=f"""Based on the research findings, write a comprehensive 
        blog post about {topic}.
        
        The blog post should:
        1. Have an engaging introduction
        2. Include 5-7 main sections with clear headings
        3. Provide practical examples
        4. Include actionable takeaways
        5. End with a compelling conclusion
        
        Target length: 1500-2000 words""",
        agent=writer,
        expected_output="A well-structured blog post in markdown format"
    )
    
    # Editing Task
    editing_task = Task(
        description="""Review and edit the blog post for:
        
        1. Grammar and spelling errors
        2. Clarity and readability
        3. Consistent tone and style
        4. Logical flow of ideas
        5. Factual accuracy
        
        Make necessary improvements and corrections.""",
        agent=editor,
        expected_output="An edited, polished version of the blog post"
    )
    
    # SEO Task
    seo_task = Task(
        description="""Optimize the blog post for SEO:
        
        1. Add relevant keywords naturally
        2. Optimize headings and subheadings
        3. Add meta description (150-160 chars)
        4. Suggest internal linking opportunities
        5. Ensure proper heading hierarchy (H1, H2, H3)
        
        Provide the final SEO-optimized version.""",
        agent=seo_specialist,
        expected_output="SEO-optimized blog post with meta description and keywords"
    )
    
    return [research_task, writing_task, editing_task, seo_task]

print("Task creation function defined")

## 4. Create and Run the Crew

In [ ]:
def run_content_crew(topic: str):
    """Run the content creation crew."""
    
    print(f"\n{'='*80}")
    print(f"Creating content about: {topic}")
    print(f"{'='*80}\n")
    
    # Create tasks
    tasks = create_content_tasks(topic)
    
    # Create crew
    crew = Crew(
        agents=[researcher, writer, editor, seo_specialist],
        tasks=tasks,
        process=Process.sequential,  # Tasks run sequentially
        verbose=True
    )
    
    # Run the crew
    result = crew.kickoff()
    
    return result

# Test the crew
topic = "The Future of AI in Healthcare"
result = run_content_crew(topic)

print("\n" + "="*80)
print("FINAL OUTPUT")
print("="*80 + "\n")
display(Markdown(str(result)))

## 5. Hierarchical Process

Let's create a crew with a manager agent for better coordination.

In [ ]:
# Create manager agent
manager = Agent(
    role='Content Manager',
    goal='Coordinate the content creation team and ensure high-quality output',
    backstory="""You are an experienced content manager who coordinates a 
    team of researchers, writers, editors, and SEO specialists. You ensure 
    that all team members work efficiently and produce exceptional content.""",
    llm=llm,
    verbose=True,
    allow_delegation=True
)

# Create hierarchical crew
def run_hierarchical_crew(topic: str):
    tasks = create_content_tasks(topic)
    
    crew = Crew(
        agents=[researcher, writer, editor, seo_specialist],
        tasks=tasks,
        process=Process.hierarchical,
        manager_llm=llm,
        verbose=True
    )
    
    return crew.kickoff()

print("Hierarchical crew setup complete")

## 6. Building a Marketing Campaign Crew

In [ ]:
# Marketing agents
market_researcher = Agent(
    role='Market Researcher',
    goal='Analyze market trends and competitor strategies',
    backstory="You are a market research expert who understands consumer behavior and market dynamics.",
    llm=llm,
    verbose=True
)

copywriter = Agent(
    role='Copywriter',
    goal='Create compelling marketing copy',
    backstory="You are a creative copywriter who crafts persuasive marketing messages.",
    llm=llm,
    verbose=True
)

social_media_manager = Agent(
    role='Social Media Manager',
    goal='Create engaging social media content',
    backstory="You are a social media expert who knows how to engage audiences across platforms.",
    llm=llm,
    verbose=True
)

# Marketing tasks
market_research_task = Task(
    description="Analyze the market for AI productivity tools",
    agent=market_researcher,
    expected_output="Market analysis report with insights and opportunities"
)

copy_task = Task(
    description="Create marketing copy for an AI productivity tool launch",
    agent=copywriter,
    expected_output="Marketing copy including headline, benefits, and call-to-action"
)

social_task = Task(
    description="Create social media posts for the product launch",
    agent=social_media_manager,
    expected_output="5 social media posts for different platforms"
)

# Create marketing crew
marketing_crew = Crew(
    agents=[market_researcher, copywriter, social_media_manager],
    tasks=[market_research_task, copy_task, social_task],
    process=Process.sequential,
    verbose=True
)

print("Marketing crew created!")

## 7. Run Marketing Campaign

In [ ]:
# Run the marketing crew
marketing_result = marketing_crew.kickoff()

print("\nMarketing Campaign Output:")
display(Markdown(str(marketing_result)))

## 8. Custom Tool Integration

In [ ]:
from crewai_tools import tool

@tool("Keyword Research Tool")
def keyword_research(topic: str) -> str:
    """Research keywords for a given topic."""
    # Simulated keyword research
    keywords = [
        f"{topic} guide",
        f"best {topic}",
        f"{topic} tutorial",
        f"how to {topic}",
        f"{topic} tips"
    ]
    return f"Top keywords for {topic}: {', '.join(keywords)}"

@tool("Competitor Analysis Tool")
def competitor_analysis(industry: str) -> str:
    """Analyze competitors in the industry."""
    return f"""Competitor analysis for {industry}:
    - Top competitors identified
    - Market share analysis
    - Strength and weaknesses
    - Opportunities for differentiation
    """

# Create agent with tools
seo_agent_with_tools = Agent(
    role='SEO Specialist with Tools',
    goal='Perform advanced SEO research and optimization',
    backstory="You are an SEO expert with access to specialized tools.",
    tools=[keyword_research, competitor_analysis],
    llm=llm,
    verbose=True
)

print("Agent with custom tools created!")

## 9. Monitoring Crew Performance

In [ ]:
class CrewMonitor:
    """Monitor and track crew performance."""
    
    def __init__(self):
        self.executions = []
    
    def track_execution(self, crew_name: str, topic: str, result: str):
        """Track a crew execution."""
        self.executions.append({
            "crew": crew_name,
            "topic": topic,
            "result_length": len(str(result)),
            "timestamp": "now"
        })
    
    def get_stats(self):
        """Get execution statistics."""
        return {
            "total_executions": len(self.executions),
            "average_result_length": sum(e["result_length"] for e in self.executions) / len(self.executions) if self.executions else 0
        }

monitor = CrewMonitor()
print("Crew monitor initialized")

## 10. Best Practices for Multi-Agent Systems

### Key Considerations:

1. **Clear Roles**: Define distinct responsibilities for each agent
2. **Task Dependencies**: Order tasks logically
3. **Communication**: Enable delegation when appropriate
4. **Quality Control**: Include review/editing agents
5. **Tools**: Equip agents with relevant tools
6. **Monitoring**: Track performance and outputs

## Next Steps

- Build autonomous agents for complex workflows
- Create interactive Streamlit applications
- Explore multimodal capabilities with Gemini

---

## Learn More

Master multi-agent systems with the **[Gen AI Crash Course](https://www.buildfastwithai.com/genai-course)** by Build Fast with AI!

**Created by [Build Fast with AI](https://www.buildfastwithai.com)**